In [5]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:

import pandas as pd
import numpy as np
import json
import openai
from sklearn.metrics.pairwise import cosine_similarity


In [7]:

CHUNK_SIZE = 600
OVERLAP = 20
openai.api_key = input("Paste your OpenAI API key here and hit enter:");
###

Paste your OpenAI API key here and hit enter:XXX


In [8]:

# Load the Kung Fu Panda script from a CSV file
df = pd.read_csv("KFP1Script.csv", encoding="ISO-8859-1")

In [17]:
df["x"]

0                                        Film title card
1      Written here is the full transcript of DreamWo...
2                                                    NaN
3      [A mysterious warrior, with his identity hidde...
4      NARRATOR (PO): [In voiceover] Legend tells of ...
                             ...                        
896    PO: [Relieved.] Oh... So, um, I should... stop...
897                                  SHIFU: If you can. 
898               PO: You want to get something to eat? 
899    [Shifu opens his eyes and sighs deeply. For a ...
900                                        SHIFU: Yeah. 
Name: x, Length: 901, dtype: object

In [13]:
df.shape

(901, 1)

In [18]:
text_list = df['x'].str.cat(sep=' ').split()
chunks = [text_list[i:i+CHUNK_SIZE] for i in range(0, len(text_list), CHUNK_SIZE-OVERLAP)]

In [19]:
df = pd.DataFrame(columns=['chunk', 'gpt_raw', 'embedding'])
for chunk in chunks:
    f = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=" ".join(chunk),
    )
    df.loc[len(df.index)] = (chunk, f, np.array(f['data'][0]['embedding']))

query = "What is Po's favorite food?"
f = openai.Embedding.create(
    model="text-embedding-ada-002",
    input=query
)
query_embedding = np.array(f['data'][0]['embedding'])

similarity = []
for arr in df['embedding'].values:
    similarity.extend(cosine_similarity(query_embedding.reshape(1, -1), arr.reshape(1, -1)))
context_chunk = chunks[np.argmax(similarity)]

query_to_send = "CONTEXT: " + " ".join(context_chunk) + "\n\n" + query
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= query_to_send,
  max_tokens=100,
  temperature=0
)
print(response['choices'][0]['text'].strip())

Po's favorite food is noodles. He is a noodle-loving panda, and his father runs a noodle shop. He often dreams of noodles, and his father even gave him a noodle hat to signify his destiny of taking over the noodle shop.
